<a href="https://colab.research.google.com/github/giovanni-lucarelli/basket/blob/main/DataWrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
# url = "https://raw.githubusercontent.com/giovanni-lucarelli/basket/main/data/games.csv"
# df = pd.read_csv(url)

df = pd.read_csv("games.csv")

df.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


In [3]:
# Drop rows containing (at least) one nan
df = df.dropna()

# Group by GAME_ID and keep only the first row for each group
df = df.groupby("GAME_ID").first().reset_index()

# Drop some unuseful columns
df = df.drop(columns=['GAME_STATUS_TEXT','TEAM_ID_home','TEAM_ID_away', "SEASON"])
df.columns

Index(['GAME_ID', 'GAME_DATE_EST', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
       'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home',
       'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away',
       'AST_away', 'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

Create new dataframe sorted by game date ("GAME_DATE_EST)

In [10]:
# Convert GAME_DATE_EST to datetime
df['GAME_DATE_EST'] = pd.to_datetime(df['GAME_DATE_EST'])

# Sort the dataframe by GAME_DATE_EST
df_sorted = df.sort_values(by='GAME_DATE_EST')

df_sorted.head()

,GAME_ID,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,10300001,2003-10-05,1610612762,1610612742,90.0,0.457,0.735,0.143,23.0,41.0,85.0,0.447,0.500,0.250,20.0,38.0,1
1,10300002,2003-10-06,1610612763,1610612749,105.0,0.494,0.618,0.267,25.0,48.0,94.0,0.427,0.700,0.154,20.0,43.0,1
7,10300009,2003-10-07,1610612758,1610612746,101.0,0.467,0.871,0.444,19.0,39.0,82.0,0.368,0.609,0.364,13.0,50.0,1
6,10300008,2003-10-07,1610612756,1610612751,79.0,0.333,0.781,0.133,17.0,43.0,101.0,0.464,0.679,0.250,22.0,47.0,0
5,10300007,2003-10-07,1610612748,1610612755,86.0,0.352,0.647,0.222,15.0,55.0,79.0,0.329,0.897,0.143,7.0,44.0,1


Building the function that computes the stats for the previous 5 games for a fixed game ("GAME_ID"), for a fixed date ("GAME_DATE_EST") for a fixed team ("HOME_TEAM_ID").

For each build a new row containing:

- [x] `WC_last5`: win count last five games
- [x] `D_PTS_last5`: (average of) delta points last five games
- [x] `D_REB_last5`: (average of) delta rebounds last five game
- [x] `D_AST_last5`: (average of) delta assists last five game
- [x] `FT_PCT_last5`: (average of) % of free throws last five game
- [x] `FG_PCT_last5`: (average of) % of field goals last five game
- [x] `FG3_PCT_last5`: (average of) % of three-point field goals last five game

**Remark:** this should be done for both home and away team

## Example with team id "1610612737" and game id "22200468"

In [11]:
game_id = 	22200468
home_team_id = 1610612737
game_date = "2022-12-21"

# Find the 5 previous games for the same home team
previous_games = df_sorted[
(df_sorted['GAME_DATE_EST'] <= game_date) &
((df_sorted['HOME_TEAM_ID'] == home_team_id) | (df_sorted['VISITOR_TEAM_ID'] == home_team_id)) &
(df_sorted['GAME_ID'] != game_id)
].sort_values(by='GAME_DATE_EST', ascending=False).head(5)

previous_games


,GAME_ID,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
24894,22200454,2022-12-19,1610612737,1610612753,126.0,0.511,0.900,0.375,29.0,41.0,125.0,0.474,0.920,0.353,29.0,45.0,1
24868,22200428,2022-12-16,1610612766,1610612737,106.0,0.398,0.684,0.349,26.0,43.0,125.0,0.547,0.824,0.517,25.0,41.0,0
24856,22200416,2022-12-14,1610612753,1610612737,135.0,0.505,0.833,0.444,28.0,46.0,124.0,0.500,0.889,0.343,30.0,39.0,1
24844,22200404,2022-12-12,1610612763,1610612737,128.0,0.602,0.833,0.406,36.0,49.0,103.0,0.356,0.773,0.324,20.0,39.0,1
24840,22200400,2022-12-11,1610612737,1610612741,123.0,0.480,0.762,0.333,28.0,45.0,122.0,0.480,0.826,0.259,28.0,53.0,1


In [12]:
# Calculate the difference in days
previous_games['days_passed'] = (pd.to_datetime(game_date) - previous_games['GAME_DATE_EST']).dt.days

# Get the minimum days passed
min_days_passed = previous_games['days_passed'].min()

min_days_passed

2

In [6]:
# Number of wins over last 5 games
previous_games_wc = []
for prev_index, prev_row in previous_games.iterrows():
  if prev_row['HOME_TEAM_ID'] == home_team_id:
    previous_games_wc.append(prev_row['HOME_TEAM_WINS'])
  else:
    previous_games_wc.append(1 - prev_row['HOME_TEAM_WINS'])

WC_last5 = sum(previous_games_wc) if len(previous_games_wc)==5 else None

WC_last5

3

In [7]:
#(average of) delta points last five games
previous_games_dpts = []
for prev_index, prev_row in previous_games.iterrows():
  if prev_row['HOME_TEAM_ID'] == home_team_id:
    previous_games_dpts.append(prev_row['PTS_home']-prev_row['PTS_away'])
  else:
    previous_games_dpts.append(prev_row['PTS_away']-prev_row['PTS_home'])

D_PTS_last5 = sum(previous_games_dpts) / len(previous_games_dpts) if len(previous_games_dpts)==5 else None

D_PTS_last5

-3.0

In [8]:
# (average of) delta rebounds last five game
previous_games_dreb = []
for prev_index, prev_row in previous_games.iterrows():
  if prev_row['HOME_TEAM_ID'] == home_team_id:
    previous_games_dreb.append(prev_row['REB_home']-prev_row['REB_away'])
  else:
    previous_games_dreb.append(prev_row['REB_away']-prev_row['REB_home'])

D_REB_last5 = sum(previous_games_dreb) / len(previous_games_dreb) if len(previous_games_dreb)==5 else None

D_REB_last5

-6.2

In [9]:
# (average of) delta assists last five game
previous_games_dast = []
for prev_index, prev_row in previous_games.iterrows():
  if prev_row['HOME_TEAM_ID'] == home_team_id:
    previous_games_dast.append(prev_row['AST_home']-prev_row['AST_away'])
  else:
    previous_games_dast.append(prev_row['AST_away']-prev_row['AST_home'])

D_AST_last5 = sum(previous_games_dast) / len(previous_games_dast) if len(previous_games_dast)==5 else None

D_AST_last5

-3.0

In [10]:
# (average of) % of free throws last five game

previous_games_ftpct = []
for prev_index, prev_row in previous_games.iterrows():
  if prev_row['HOME_TEAM_ID'] == home_team_id:
    previous_games_ftpct.append(prev_row['FT_PCT_home'])
  else:
    previous_games_ftpct.append(prev_row['FT_PCT_away'])

FT_PCT_last5 = sum(previous_games_ftpct) / len(previous_games_ftpct) if len(previous_games_ftpct)==5 else None

FT_PCT_last5

0.8295999999999999

In [11]:
# (average of) % of field goals last five game

previous_games_fgpct = []
for prev_index, prev_row in previous_games.iterrows():
  if prev_row['HOME_TEAM_ID'] == home_team_id:
    previous_games_fgpct.append(prev_row['FG_PCT_home'])
  else:
    previous_games_fgpct.append(prev_row['FG_PCT_away'])

FG_PCT_last5 = sum(previous_games_fgpct) / len(previous_games_fgpct) if len(previous_games_fgpct)==5 else None

FG_PCT_last5

0.4788

In [12]:
# (average of) % of three-point field goals last five game

previous_games_fg3pct = []
for prev_index, prev_row in previous_games.iterrows():
  if prev_row['HOME_TEAM_ID'] == home_team_id:
    previous_games_fg3pct.append(prev_row['FG3_PCT_home'])
  else:
    previous_games_fg3pct.append(prev_row['FG3_PCT_away'])

FG3_PCT_last5 = sum(previous_games_fg3pct) / len(previous_games_fg3pct) if len(previous_games_fg3pct)==5 else None

FG3_PCT_last5

0.3784

## `compute_stats` function

The function `compute_stats(previous_games, home_team_id)` takes as input a (small) dataframe `previous_games` (thanks to `df_previous5(df_sorted, game_id, home_team_id, game_date)`) and a team id and returns the previously described statistics of that team in the five previous days of the championship.

**Remark:** the dataframe provided to `df_previous5` should be sorted by date (`GAME_DATE_EST`)!

In [16]:
def df_previous5(df_sorted, game_id, home_team_id, game_date):
  previous_games = df_sorted[
    (df_sorted['GAME_DATE_EST'] <= game_date) &
    ((df_sorted['HOME_TEAM_ID'] == home_team_id) | (df_sorted['VISITOR_TEAM_ID'] == home_team_id)) &
    (df_sorted['GAME_ID'] != game_id)
    ].sort_values(by='GAME_DATE_EST', ascending=False).head(5)
  
  # Calculate the difference in days
  previous_games['days_passed'] = (pd.to_datetime(game_date) - previous_games['GAME_DATE_EST']).dt.days

  return previous_games

In [17]:
game_id = 	22200468
home_team_id = 1610612737
game_date = "2022-12-21"

df_previous5(df_sorted, game_id, home_team_id, game_date)

,GAME_ID,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,days_passed
24894,22200454,2022-12-19,1610612737,1610612753,126.0,0.511,0.900,0.375,29.0,41.0,125.0,0.474,0.920,0.353,29.0,45.0,1,2
24868,22200428,2022-12-16,1610612766,1610612737,106.0,0.398,0.684,0.349,26.0,43.0,125.0,0.547,0.824,0.517,25.0,41.0,0,5
24856,22200416,2022-12-14,1610612753,1610612737,135.0,0.505,0.833,0.444,28.0,46.0,124.0,0.500,0.889,0.343,30.0,39.0,1,7
24844,22200404,2022-12-12,1610612763,1610612737,128.0,0.602,0.833,0.406,36.0,49.0,103.0,0.356,0.773,0.324,20.0,39.0,1,9
24840,22200400,2022-12-11,1610612737,1610612741,123.0,0.480,0.762,0.333,28.0,45.0,122.0,0.480,0.826,0.259,28.0,53.0,1,10


In [18]:
def compute_stats(previous_games, home_team_id):

    results = []

    # Number of wins over last 5 games
    previous_games_wc = []
    for prev_index, prev_row in previous_games.iterrows():
      if prev_row['HOME_TEAM_ID'] == home_team_id:
        previous_games_wc.append(prev_row['HOME_TEAM_WINS'])
      else:
        previous_games_wc.append(1 - prev_row['HOME_TEAM_WINS'])

    WC_last5 = sum(previous_games_wc) if len(previous_games_wc)==5 else None

    #(average of) delta points last five games
    previous_games_dpts = []
    for prev_index, prev_row in previous_games.iterrows():
      if prev_row['HOME_TEAM_ID'] == home_team_id:
        previous_games_dpts.append(prev_row['PTS_home']-prev_row['PTS_away'])
      else:
        previous_games_dpts.append(prev_row['PTS_away']-prev_row['PTS_home'])

    D_PTS_last5 = sum(previous_games_dpts) / len(previous_games_dpts) if len(previous_games_dpts)==5 else None

    # (average of) delta rebounds last five game
    previous_games_dreb = []
    for prev_index, prev_row in previous_games.iterrows():
      if prev_row['HOME_TEAM_ID'] == home_team_id:
        previous_games_dreb.append(prev_row['REB_home']-prev_row['REB_away'])
      else:
        previous_games_dreb.append(prev_row['REB_away']-prev_row['REB_home'])

    D_REB_last5 = sum(previous_games_dreb) / len(previous_games_dreb) if len(previous_games_dreb)==5 else None

    # (average of) delta assists last five game
    previous_games_dast = []
    for prev_index, prev_row in previous_games.iterrows():
      if prev_row['HOME_TEAM_ID'] == home_team_id:
        previous_games_dast.append(prev_row['AST_home']-prev_row['AST_away'])
      else:
        previous_games_dast.append(prev_row['AST_away']-prev_row['AST_home'])

    D_AST_last5 = sum(previous_games_dast) / len(previous_games_dast) if len(previous_games_dast)==5 else None

    # (average of) % of free throws last five game

    previous_games_ftpct = []
    for prev_index, prev_row in previous_games.iterrows():
      if prev_row['HOME_TEAM_ID'] == home_team_id:
        previous_games_ftpct.append(prev_row['FT_PCT_home'])
      else:
        previous_games_ftpct.append(prev_row['FT_PCT_away'])

    FT_PCT_last5 = sum(previous_games_ftpct) / len(previous_games_ftpct) if len(previous_games_ftpct)==5 else None

    # (average of) % of field goals last five game

    previous_games_fgpct = []
    for prev_index, prev_row in previous_games.iterrows():
      if prev_row['HOME_TEAM_ID'] == home_team_id:
        previous_games_fgpct.append(prev_row['FG_PCT_home'])
      else:
        previous_games_fgpct.append(prev_row['FG_PCT_away'])

    FG_PCT_last5 = sum(previous_games_fgpct) / len(previous_games_fgpct) if len(previous_games_fgpct)==5 else None

    # (average of) % of three-point field goals last five game

    previous_games_fg3pct = []
    for prev_index, prev_row in previous_games.iterrows():
      if prev_row['HOME_TEAM_ID'] == home_team_id:
        previous_games_fg3pct.append(prev_row['FG3_PCT_home'])
      else:
        previous_games_fg3pct.append(prev_row['FG3_PCT_away'])

    FG3_PCT_last5 = sum(previous_games_fg3pct) / len(previous_games_fg3pct) if len(previous_games_fg3pct)==5 else None

    

    results.append({
            'WC_last5': WC_last5,
            'D_PTS_last5' : D_PTS_last5,
            'D_REB_last5': D_REB_last5,
            'D_AST_last5' : D_AST_last5,
            'FT_PCT_last5' : FT_PCT_last5,
            'FG_PCT_last5' : FG_PCT_last5,
            'FG3_PCT_last5' : FG3_PCT_last5,
            # 'days_last_match' : min_days_passed
        })

    return pd.DataFrame(results)

## Building the final dataframe

In [19]:
# sorting the original dataframe by date
df_sorted = df.sort_values(by='GAME_DATE_EST')
df_sorted.tail()

,GAME_ID,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
24909,22200469,2022-12-21,1610612738,1610612754,112.0,0.386,0.840,0.317,26.0,62.0,117.0,0.469,0.778,0.462,27.0,47.0,0
24908,22200468,2022-12-21,1610612737,1610612741,108.0,0.429,1.000,0.378,22.0,47.0,110.0,0.500,0.773,0.292,20.0,47.0,0
24911,22200471,2022-12-21,1610612752,1610612761,106.0,0.553,0.611,0.423,25.0,38.0,113.0,0.447,0.909,0.265,17.0,38.0,0
24918,22200478,2022-12-22,1610612762,1610612764,120.0,0.488,0.952,0.457,16.0,40.0,112.0,0.561,0.765,0.333,20.0,37.0,1
24917,22200477,2022-12-22,1610612740,1610612759,126.0,0.484,0.926,0.382,25.0,46.0,117.0,0.478,0.815,0.321,23.0,44.0,1


In [20]:
# Initialize an empty list to store the dataframes
all_home_away_stats = []

for index, row in df_sorted.iterrows():

  game_id = row['GAME_ID']
  game_date = row['GAME_DATE_EST']
  home_team_id = row['HOME_TEAM_ID']
  away_team_id = row['VISITOR_TEAM_ID']

  home_wins = row['HOME_TEAM_WINS']

  # compute last 5 game df for the home team
  previous_games_home = df_previous5(df_sorted, game_id, home_team_id, game_date)

  # compute stats for the home team
  stats_home = compute_stats(previous_games_home, home_team_id)

  # rename the column for home stats
  stats_home = stats_home.rename(columns={
      'WC_last5': 'home_WC_last5',
      'D_PTS_last5': 'home_D_PTS_last5',
      'D_REB_last5': 'home_D_REB_last5',
      'D_AST_last5': 'home_D_AST_last5',
      'FT_PCT_last5': 'home_FT_PCT_last5',
      'FG_PCT_last5': 'home_FG_PCT_last5',
      'FG3_PCT_last5': 'home_FG3_PCT_last5'
  })


  # compute last 5 game df for the away team
  previous_games_away = df_previous5(df_sorted, game_id, away_team_id, game_date)

  # compute stats for the away team
  stats_away = compute_stats(previous_games_away, away_team_id)

  # rename the column for away stats
  stats_away = stats_away.rename(columns={
      'WC_last5': 'away_WC_last5',
      'D_PTS_last5': 'away_D_PTS_last5',
      'D_REB_last5': 'away_D_REB_last5',
      'D_AST_last5': 'away_D_AST_last5',
      'FT_PCT_last5': 'away_FT_PCT_last5',
      'FG_PCT_last5': 'away_FG_PCT_last5',
      'FG3_PCT_last5': 'away_FG3_PCT_last5'
  })

  # Concatenate home and away stats
  home_away_stats = pd.concat([stats_home, stats_away], axis=1)

  # add the home days passed
  home_days_passed = previous_games_home['days_passed'].min()
  home_away_stats['home_days_passed'] = home_days_passed
  
  # add the away days passed
  away_days_passed = previous_games_away['days_passed'].min()
  home_away_stats['away_days_passed'] = away_days_passed

  # add the home_wins
  home_away_stats['HOME_WINS'] = home_wins

  # add the date of the game
  home_away_stats['GAME_DATE_EST'] = game_date

  # Instead of just printing, append the dataframe to the list
  all_home_away_stats.append(home_away_stats)

# After the loop, concatenate all the dataframes in the list
final_df = pd.concat(all_home_away_stats, ignore_index=True)
final_df

/tmp/ipykernel_4767/1383323412.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_home_away_stats, ignore_index=True)


,home_WC_last5,home_D_PTS_last5,home_D_REB_last5,home_D_AST_last5,home_FT_PCT_last5,home_FG_PCT_last5,home_FG3_PCT_last5,away_WC_last5,away_D_PTS_last5,away_D_REB_last5,away_D_AST_last5,away_FT_PCT_last5,away_FG_PCT_last5,away_FG3_PCT_last5,home_days_passed,away_days_passed,HOME_WINS,GAME_DATE_EST
0,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2003-10-05
1,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2003-10-06
2,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2003-10-07
3,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2003-10-07
4,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2003-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26518,1,-8.6,-5.8,0.8,0.7584,0.4156,0.2842,1,-2.2,-10.0,5.0,0.7774,0.4742,0.3634,3.0,3.0,0,2022-12-21
26519,3,-3.0,-6.2,-3.0,0.8296,0.4788,0.3784,1,-9.2,-5.0,-0.4,0.8296,0.5134,0.3640,2.0,1.0,0,2022-12-21
26520,5,17.0,13.0,-2.2,0.7912,0.4666,0.3754,0,-7.0,-3.8,-4.8,0.8310,0.4458,0.3140,1.0,2.0,0,2022-12-21
26521,3,-2.0,-4.8,0.0,0.6698,0.4556,0.3542,1,-6.6,-4.6,-0.6,0.7330,0.4724,0.3730,2.0,2.0,1,2022-12-22


We also add data for test/train division

## Save new csv file

In [21]:
# drop the rows that contain at least one NaN: it means there are not enough games played before by the home team or by the away team
final_df = final_df.dropna()

In [22]:
# Save the final dataframe to a new CSV file
final_df.to_csv('games_processed_days.csv', index=False)